In [132]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import matplotlib as mplib
import matplotlib.pyplot as plt # visualization package 1
#import seaborn as sns           # visualization package 2
from datetime import datetime
import copy
import ast
from collections import Counter
import itertools
#import nltk
from ast import literal_eval

#clustering
#from sklearn.cluster import KMeans
#from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import silhouette_score

import time
from collections import Counter
import timeit
    
    

# so images get plotted in the notebook
%matplotlib inline

In [2]:
#output a directory for each patent's city and ipc section matched with its index number so that we can use the 
#index number to extract rows from dataframe for each city and ipc section
def get_directory(df):
    for i in range(len(df)):
        ipc = literal_eval(df['IPCs'][i])
        for j in range(len(ipc)):  
            sec = ipc[j]['ipc_section']
            try:
                asgn = literal_eval(df['assignees'][i])
                for k in range(len(asgn)):
                    yield (i, ((asgn[k]['assignee_city'], asgn[k]['assignee_state']), sec))
            except SyntaxError:
                asgn = df.iloc[i]['assignees']
                for k in range(asgn.count('city')):
                    yield (i, ((asgn.split(':')[2+k*3].split(',')[0][3:-1], asgn[k]['assignee_state']), sec))

In [3]:
#output a directory for each patent's city and ipc section matched with its index number so that we can use the 
#index number to extract rows from dataframe for each city and ipc section
def get_inv_directory(curr_df):
    df = curr_df.reset_index()
    for i in range(len(df)):
        ipc = literal_eval(df['IPCs'][i])
        for j in range(len(ipc)):  
            sec = ipc[j]['ipc_section']
            try:
                asgn = literal_eval(df['inventors'][i])
                for k in range(len(asgn)):
                    yield (i, ((asgn[k]['inventor_city'], asgn[k]['inventor_state']), sec))
            except SyntaxError:
                asgn = df.iloc[i]['inventors']
                for k in range(asgn.count('city')):
                    yield (i, ((asgn.split(':')[2+k*3].split(',')[0][3:-1],asgn[k]['inventor_state']) , sec))

In [162]:
#output a list of tuples with unique values (city or ipc section) and corresponding number of patents
#num: 0-city, 1-ipc section
def count_unique(directory, num):
    counts = {}
    for tup in directory:
        counts[tup[1][num]] = counts.get(tup[1][num], 0) + 1
    return sorted(counts.items(), key=lambda x: x[1], reverse=True)

In [5]:
#df: compete dataframe with all patents
#directory: directory we get from above
#num: 0-city, 1-ipc section
#string: city or ipc section that we want to refine
def df_by_index(df, directory, num, string):
    return df.iloc[list(map(lambda x: x[0], filter(lambda x: x[1][num]==string, directory)))]

In [6]:
#count sections for the called city
def count_sections(curr_city, city_directory, section):
    temp_dir = [x for x in city_directory if x[1][0] == curr_city]
    temp = count_unique(temp_dir, 1)
    to_return = [x[1] for x in temp if x[0] == section]
    if len(to_return) == 1:
        return(to_return[0])
    else:
        return(0)

#to find IPC section distribution for inventor cities
def count_cities_by_section(df, city_directory):
    sections = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    for i in sections:
        df[i] = df.city.apply(lambda x: count_sections(x, city_directory, i))
    df['sum'] = df.sum(axis=1)

In [7]:
def get_assignee_type(x):
    to_get = literal_eval(x)
    assignee_type = to_get[0]['assignee_type']
    return(assignee_type)

In [133]:
def count_IPCs(x, sections):
    to_return = literal_eval(x)
    for i in range(len(to_return)):
        sections.append(to_return[i]['ipc_section'])

In [177]:
#This function creates a dictionary that contains the top 10 Assignee cities
# Each key is a city. Within the city key is another dictionary containing IPC dist, location and inventor cities
def create_dict_of_cities(cities, dict_to_add, curr_df, assignee_dir, inventor_dir, year):
    start_total = time.time()
    for i in range(len(cities)):
        
        #dict_city is overarching dictionary for the current Assignee city
        #dict_inv is the dictionary for inventions from current Assignee city
        dict_city = {}
        dict_to_add[i] = dict_city
        city = cities[i][0]
        state = cities[i][1]
        dict_city['City'] = city
        dict_city['State'] = state
        dict_city['city_state'] = cities[i]
        dict_city['Year'] = year
        
        
        
        #get dataframe for current Assignee city
        df_temp = df_by_index(curr_df, assignee_dir, 0, cities[i])
        df_temp.reset_index(inplace = True)
        
        
        #dict_types = {}
        df_temp['assignee_type'] = df_temp['assignees'].apply(lambda x: get_assignee_type(x))
        for z in ['2','3','4','5','6','7','8','9']:
            key = "assignee_type_" + str(z)
            try:
                dict_city[key] = len(df_temp.loc[df_temp['assignee_type'] == z])
            except:
                dict_city[key] = 0
        
        #add total number of patents for a city
        num_patents = len(df_temp)
        dict_city["Patents"] = num_patents

        #Get directories and cities and sections for current city
        temp_directory = sorted(list(set(get_directory(df_temp))), key=lambda x: x[0])
        temp_sections = count_unique(temp_directory, 1)
        
        #Get inventor info for current city
        df_inv = df_by_index(curr_df, inventor_dir, 0, cities[i])
        num_inv_patents = len(df_inv)
        dict_city['inventor_patents'] = num_inv_patents
        
        #create patent_type_dict
        patent_types = ['utility', 'design', 'reissue', 'plant', 'statutory invention registration']
        values = df_temp.patent_type.value_counts()
        for j in patent_types:
            key = "pt_" + str(j)
            try:
                dict_city[key] = values[j]/num_patents
            except:
                dict_city[key] = 0
                
        
        section_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
        sections = []
        inv_sections = []
        placeholder = df_temp.IPCs.apply(lambda x: count_IPCs(x, sections))
        placeholder = df_inv.IPCs.apply(lambda x: count_IPCs(x, inv_sections))
        total = len(sections)
        total_inv = len(inv_sections)
        count = Counter(sections)
        inv_count = Counter(inv_sections)
        for k in section_names:
            key = "assignee_IPC_" + str(k)
            try:
                dict_city[key] = count[k]/total
            except:
                dict_city[key] = 0
            
            inv_key = 'inventor_IPC_' + str(k)
            try:
                dict_city[inv_key] = inv_count[k]/total_inv
            except:
                dict_city[inv_key] = 0
        
        
        #create a dictionary for IPCs sections in inventor cities
        #inv_temp_directory = sorted(list(set(get_inv_directory(df_inv))), key=lambda x: x[0])
        #inv_sections = count_unique(inv_temp_directory, 1)
        #section_inv_values = [x[1] for x in inv_sections if x[0] in sections]
        #section_inv_names = [x[0] for x in inv_sections if x[0] in sections]
        #for k in range(len(section_inv_names)):
        #    key = "inventor_IPC_" + str(section_inv_names[k])
        ##    try:
         #       dict_city[key] = (section_inv_values[k])/num_inv_patents
         #   except:
         #       dict_city[key] = 0
        
        
        #find number of inventor city mentions for curr_city
        try:
            dict_city['Inv_to_Assignee_ratio'] = num_inv_patents / num_patents
        except:
            dict_city['Inv_to_Assignee_ratio'] = 0
        
        #find total times patents have been cited in assignee and inventor city
        dict_city['assignee_pats_cited'] = int(df_temp.patent_num_foreign_citations.sum() + df_temp.patent_num_us_application_citations.sum())
        dict_city['inventor_pats_cited'] = int(df_inv.patent_num_foreign_citations.sum() + df_inv.patent_num_us_application_citations.sum())
        try:
            dict_city['assignee_pats_cited_ratio'] = dict_city['assignee_pats_cited'] / num_patents
        except:
            dict_city['assignee_pats_cited_ratio'] = 0
        try:
            dict_city['inventor_pats_cited_ratio'] = dict_city['inventor_pats_cited'] / num_inv_patents
        except:
            dict_city['inventor_pats_cited_ratio'] = 0

        if i % 200 == 0:
            print(time.time())
            print(str(i) + " of " + str(len(cities)))
    end_total = time.time()
    print("Total time : " + str(end_total-start_total))
        

In [178]:
directory_dict = {}
top_cities_by_year = {}
top_sections_by_year = {}
inv_directory_dict = {}
inv_top_cities_by_year = {}

In [179]:
#create curr year dicts
year = 1976
file_name = '/Volumes/Rohun Backups/Datasets/Capstone/Patents_by_year/patents_' + str(year) + '.csv'
df_curr = pd.read_csv(file_name)
print("Year: " + str(year) + " Number of patents: " + str(len(df_curr)))


dict_dir = sorted(list(set(get_directory(df_curr))), key=lambda x: x[0])
print('done with dict dir')
dict_inv_dir = sorted(list(set(get_inv_directory(df_curr))), key=lambda x: x[0])
print('done with inv dir')
top_cities = count_unique(dict_dir, 0)
print('done with top cities')
top_inv_cities = count_unique(dict_inv_dir, 0)
print("Dictionaries loaded")

Year: 1976 Number of patents: 34796
done with dict dir
done with inv dir
done with top cities
Dictionaries loaded


In [180]:
assignees_dict = {}
top_cities = top_cities[:1000]
top_cities_names = [x[0] for x in top_cities]
create_dict_of_cities(top_cities_names, assignees_dict, df_curr, dict_dir, dict_inv_dir, year)

/Users/rohun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1555875806.789396
0 of 1000
1555875841.108017
200 of 1000
1555875871.193873
400 of 1000
1555875900.653567
600 of 1000
1555875930.063411
800 of 1000
Total time : 153.32476687431335


In [181]:
cities_df = []
for key in assignees_dict:
    temp = json_normalize(assignees_dict[key])
    cities_df.append(temp)

In [182]:
trial = pd.concat(cities_df, ignore_index = True)

In [183]:
trial.describe()

,Inv_to_Assignee_ratio,Patents,Year,assignee_IPC_A,assignee_IPC_B,assignee_IPC_C,assignee_IPC_D,assignee_IPC_E,assignee_IPC_F,assignee_IPC_G,...,inventor_IPC_G,inventor_IPC_H,inventor_patents,inventor_pats_cited,inventor_pats_cited_ratio,pt_design,pt_plant,pt_reissue,pt_statutory invention registration,pt_utility
count,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.000000
mean,2.255663,36.052000,1976.0,0.126935,0.266185,0.103322,0.077172,0.049394,0.126036,0.130060,...,0.131134,0.121150,26.340000,9.183000,0.348488,0.067295,0.006587,0.007753,0.0,0.918131
std,3.411266,164.978056,0.0,0.179297,0.241489,0.182418,0.152625,0.119215,0.179187,0.180359,...,0.164823,0.157821,44.693175,15.842869,0.370380,0.144148,0.065926,0.036559,0.0,0.160658
min,0.000000,3.000000,1976.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.535119,4.000000,1976.0,0.000000,0.066204,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,5.000000,1.000000,0.100000,0.000000,0.000000,0.000000,0.0,0.900000
50%,1.129428,7.000000,1976.0,0.042339,0.228390,0.000000,0.000000,0.000000,0.047619,0.043478,...,0.081752,0.069643,12.000000,4.000000,0.277778,0.000000,0.000000,0.000000,0.0,1.000000
75%,2.459135,17.000000,1976.0,0.200000,0.400000,0.148611,0.090909,0.026779,0.198406,0.209583,...,0.206136,0.187500,30.000000,10.250000,0.467187,0.067045,0.000000,0.000000,0.0,1.000000
max,46.333333,4077.000000,1976.0,1.000000,1.000000,1.000000,1.000000,0.857143,1.000000,1.000000,...,1.000000,1.000000,557.000000,141.000000,3.250000,1.000000,1.000000,0.500000,0.0,1.000000


In [152]:
df_temp = df_by_index(df_curr, dict_dir, 0, ('Ardsley', 'NY'))
df_temp.reset_index(inplace = True)

In [153]:
def trial_func(x, sections):
    to_return = literal_eval(x)
    for i in range(len(to_return)):
        sections.append(to_return[i]['ipc_section'])

In [154]:
from collections import Counter
import timeit
sections = []

df_temp.IPCs.apply(lambda x: trial_func(x, sections))
count = Counter(sections)

In [155]:
for i in range(len(df_temp)):
    print(literal_eval(df_temp.IPCs[i]))

[{'ipc_subgroup': '30', 'ipc_sequence': '0', 'ipc_subclass': 'C', 'ipc_main_group': '39', 'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_subgroup': '32', 'ipc_sequence': '0', 'ipc_subclass': 'C', 'ipc_main_group': '149', 'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_subgroup': '32', 'ipc_sequence': '0', 'ipc_subclass': 'C', 'ipc_main_group': '149', 'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_subgroup': '22', 'ipc_sequence': '0', 'ipc_subclass': 'N', 'ipc_main_group': '9', 'ipc_section': 'A', 'ipc_class': '01'}]
[{'ipc_subgroup': '20', 'ipc_sequence': '0', 'ipc_subclass': 'N', 'ipc_main_group': '9', 'ipc_section': 'A', 'ipc_class': '01'}, {'ipc_subgroup': '78', 'ipc_sequence': '1', 'ipc_subclass': 'C', 'ipc_main_group': '121', 'ipc_section': 'C', 'ipc_class': '07'}]
[{'ipc_subgroup': '20', 'ipc_sequence': '0', 'ipc_subclass': 'N', 'ipc_main_group': '9', 'ipc_section': 'A', 'ipc_class': '01'}, {'ipc_subgroup': '78', 'ipc_sequence': '1', 'ipc_subclass': 'C', 'ipc_main_group': '121', '

In [156]:
count

Counter({'C': 536, 'A': 183, 'F': 5, 'H': 8, 'D': 56, 'B': 45, 'G': 2})